In [179]:
import requests
import dill as pickle
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
import time

with open("comp_actor_movies.pkl", 'r') as picklefile: 
    comp_actor_movies = pickle.load(picklefile)
    
col = ['date', 'date_url', 'movie', 'movie_url', 'studio', 'studio_url', 'gross', 'gross_url', 'theaters_g', 'theaters_g_url', 'opening', 'opening_url', 'theaters_o', 'theaters_o_url', 'rank', 'rank_url','name']
comp_actor_df = pd.DataFrame(comp_actor_movies, columns = col)
comp_actor_df['date'] = pd.to_datetime(comp_actor_df['date'])
comp_actor_df = comp_actor_df[comp_actor_df.date != 'N/A']
comp_actor_df['gross'] = (comp_actor_df['gross'].replace('[\$,)]','', regex=True).replace( '[(]','-', regex=True ).convert_objects(convert_numeric=True).astype(float))
comp_actor_df['name'] = comp_actor_df['name'].str.replace('*', '')
calc_comp_actor_df = comp_actor_df[['name','movie','date','gross']]
calc_comp_actor_df.set_index(['date'], inplace = True)

def gross_by_day(start, end, number_of_days, actor_frame, actor_name):
    ts = pd.DataFrame(index=pd.date_range(start, end)).sort_index(ascending = True)
    ts.index.name = 'date'
    df = actor_frame[['gross']].sort_index(ascending = True)
    df_new = pd.merge(left=ts, right=df, how='left', left_index=True, right_index=True)
    df_new['gross'].fillna(0, inplace=True)
    df_before = df_new.shift(periods=1)
    #df_after = df_new
    df_before_sum = pd.rolling_sum(df_before, number_of_days)
    df_before_sum.rename(columns={'gross':'net_gross_before'},inplace=True)
    #df_after_count = pd.rolling_count(df_after, number_of_days)
    #df_after_count.rename(columns={'gross':'total_movies_after'},inplace=True)
    df_after = df_new.shift(periods=-number_of_days)
    #df_after = df_new.sort_index(ascending = False)
    df_after_sum = pd.rolling_sum(df_after, number_of_days)
    df_after_sum.rename(columns={'gross':'net_gross_after'},inplace=True)
    #df_before_count = pd.rolling_count(df_before, number_of_days)
    #df_before_count.rename(columns={'gross':'total_movies_before'},inplace=True)
    df_final = pd.concat([df_after_sum, df_before_sum],axis=1) #df_after_count, df_before_count
    return df_final

def concat_delta_gross(name_list, total_frame):
    col = ['name', 'movie', 'date', 'gross','net_gross_after', 'net_gross_before']
    concat = pd.DataFrame(columns = col)
    concat.set_index(['date'], inplace = True)
    for name in name_list:
        actor_frame = calc_comp_actor_df[calc_comp_actor_df['name'] == name]
        single_actor_gross = gross_by_day('1/1/1950', '12/12/2015', 365, actor_frame, name)
        #actor_frame.set_index(['date'], inplace = True)
        df_new = pd.merge(left=actor_frame, right=single_actor_gross, how='left', left_index=True, right_index=True)
        concat = concat.append(df_new)
    return concat

list_of_actors = list(set(calc_comp_actor_df['name'].tolist()))    
df = concat_delta_gross(list_of_actors, calc_comp_actor_df)

In [176]:
def gross_by_day(start, end, number_of_days, actor_frame, actor_name):
    ts = pd.DataFrame(index=pd.date_range(start, end)).sort_index(ascending = True)
    ts.index.name = 'date'
    df = actor_frame[['gross']].sort_index(ascending = True)
    df_new = pd.merge(left=ts, right=df, how='left', left_index=True, right_index=True)
    df_new['gross'].fillna(0, inplace=True)
    df_before = df_new.shift(periods=1)
    #df_after = df_new
    df_before_sum = pd.rolling_sum(df_before, number_of_days)
    df_before_sum.rename(columns={'gross':'net_gross_before'},inplace=True)
    #df_after_count = pd.rolling_count(df_after, number_of_days)
    #df_after_count.rename(columns={'gross':'total_movies_after'},inplace=True)
    df_after = df_new.shift(periods=-number_of_days)
    #df_after = df_new.sort_index(ascending = False)
    df_after_sum = pd.rolling_sum(df_after, number_of_days)
    df_after_sum.rename(columns={'gross':'net_gross_after'},inplace=True)
    #df_before_count = pd.rolling_count(df_before, number_of_days)
    #df_before_count.rename(columns={'gross':'total_movies_before'},inplace=True)
    df_final = pd.concat([df_after_sum, df_before_sum],axis=1) #df_after_count, df_before_count
    return df_final

def concat_delta_gross(name_list, total_frame):
    col = ['name', 'movie', 'date', 'gross','net_gross_after', 'net_gross_before']
    concat = pd.DataFrame(columns = col)
    concat.set_index(['date'], inplace = True)
    for name in name_list:
        actor_frame = calc_comp_actor_df[calc_comp_actor_df['name'] == name]
        single_actor_gross = gross_by_day('1/1/1950', '12/12/2015', 365, actor_frame, name)
        #actor_frame.set_index(['date'], inplace = True)
        df_new = pd.merge(left=actor_frame, right=single_actor_gross, how='left', left_index=True, right_index=True)
        concat = concat.append(df_new)
    return concat

list_of_actors = list(set(calc_comp_actor_df['name'].tolist()))    
df = concat_delta_gross(list_of_actors, calc_comp_actor_df)

In [193]:
#calc_comp_actor_df = calc_comp_actor_df.set_index('date').head()
df

,name,movie,gross,net_gross_after,net_gross_before
2008-07-11,Josh Hartnett,August,12636,0,42741569
2007-10-19,Josh Hartnett,30 Days of Night,39568996,12636,3172573
2007-08-24,Josh Hartnett,Resurrecting the Champ,3172573,39581632,22545080
2006-09-15,Josh Hartnett,The Black Dahlia,22545080,3172573,22531472
2006-04-14,Josh Hartnett,Mozart and the Whale,36006,22545080,22495466
2006-04-07,Josh Hartnett,Lucky Number Slevin,22495466,22581086,0
2005-04-01,Josh Hartnett,Sin City,74103820,0,13001257
2004-09-03,Josh Hartnett,Wicker Park,13001257,74103820,0
2003-06-13,Josh Hartnett,Hollywood Homicide,30940691,0,0
2002-03-01,Josh Hartnett,40 Days and 40 Nights,37950822,0,330564478


In [148]:
actor_frame = calc_comp_actor_df[calc_comp_actor_df['name'] == 'Nicolas Cage']
#actor_frame['name'] = 'whatever'

In [71]:
#df = concat_delta_gross(['Nicolas Cage'], calc_comp_actor_df)
#df.sort()
#actor_frame.set_index('date')
#df_new = pd.merge(left=actor_frame, right=single_actor_gross, how='left', left_index=True, right_index=True)

In [208]:
comparison_nc_df = calc_comp_actor_df[calc_comp_actor_df['name'] == 'Nicolas Cage']
comparison_nc_df = comparison_nc_df['movie']
nc_dates = list(comparison_nc_df.index)

In [157]:
comparison_movie_df

,name,movie,date,gross
10751,Sean Bean,National Treasure,11/19/04,173008894
11044,Jon Voight,National Treasure,11/19/04,173008894
11217,Justin Bartha,National Treasure,11/19/04,173008894
12790,Nicolas Cage,National Treasure,11/19/04,173008894


In [189]:
comparison_actor_df = calc_comp_actor_df[calc_comp_actor_df['name'] == 'Jon Voight']
comparison_actor_df['other name'] = 'fred'
comparison_actor_df

/Users/johnwalters/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


,name,movie,gross,NC Movie,other name
date,,,,,
8/30/13,Jon Voight,Getaway (2013),10501938,False,fred
11/26/08,Jon Voight,Four Christmases,120146040,False,fred
10/24/08,Jon Voight,Pride & Glory,15740721,False,fred
10/3/08,Jon Voight,An American Carol,7013191,False,fred
12/21/07,Jon Voight,National Treasure: Book of Secrets,219964115,True,fred
8/24/07,Jon Voight,September Dawn,1066555,False,fred
8/3/07,Jon Voight,BRATZ,10010209,False,fred
7/3/07,Jon Voight,Transformers,319246193,False,fred
1/13/06,Jon Voight,Glory Road,42647449,False,fred


In [209]:
calc_comp_actor_df['NC Movie'] = calc_comp_actor_df[['movie']].isin(nc_dates)
calc_comp_actor_df

/Users/johnwalters/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,name,movie,gross,NC Movie
date,,,,
7/11/08,Josh Hartnett,August,12636,False
10/19/07,Josh Hartnett,30 Days of Night,39568996,False
8/24/07,Josh Hartnett,Resurrecting the Champ,3172573,False
9/15/06,Josh Hartnett,The Black Dahlia,22545080,False
4/14/06,Josh Hartnett,Mozart and the Whale,36006,False
4/7/06,Josh Hartnett,Lucky Number Slevin,22495466,False
4/1/05,Josh Hartnett,Sin City,74103820,False
9/3/04,Josh Hartnett,Wicker Park,13001257,False
6/13/03,Josh Hartnett,Hollywood Homicide,30940691,False


In [160]:
comparison_actor_df

,name,movie,date,gross
11035,Jon Voight,Getaway (2013),8/30/13,10501938
11036,Jon Voight,Four Christmases,11/26/08,120146040
11037,Jon Voight,Pride & Glory,10/24/08,15740721
11038,Jon Voight,An American Carol,10/3/08,7013191
11039,Jon Voight,National Treasure: Book of Secrets,12/21/07,219964115
11040,Jon Voight,September Dawn,8/24/07,1066555
11041,Jon Voight,BRATZ,8/3/07,10010209
11042,Jon Voight,Transformers,7/3/07,319246193
11043,Jon Voight,Glory Road,1/13/06,42647449
11044,Jon Voight,National Treasure,11/19/04,173008894


In [205]:
def gross_by_day(start, end, number_of_days, actor_frame, actor_name):
    ts = pd.DataFrame(index=pd.date_range(start, end)).sort_index(ascending = True)
    ts.index.name = 'date'
    df = actor_frame[['gross']].sort_index(ascending = True)
    df_new = pd.merge(left=ts, right=df, how='left', left_index=True, right_index=True)
    df_new['gross'].fillna(0, inplace=True)
    df_before = df_new.shift(periods=1)
    #df_after = df_new
    df_before_sum = pd.rolling_sum(df_before, number_of_days)
    df_before_sum.rename(columns={'gross':'net_gross_before'},inplace=True)
    #df_after_count = pd.rolling_count(df_after, number_of_days)
    #df_after_count.rename(columns={'gross':'total_movies_after'},inplace=True)
    df_after = df_new.shift(periods=-number_of_days)
    #df_after = df_new.sort_index(ascending = False)
    df_after_sum = pd.rolling_sum(df_after, number_of_days)
    df_after_sum.rename(columns={'gross':'net_gross_after'},inplace=True)
    #df_before_count = pd.rolling_count(df_before, number_of_days)
    #df_before_count.rename(columns={'gross':'total_movies_before'},inplace=True)
    df_final = pd.concat([df_after_sum, df_before_sum],axis=1) #df_after_count, df_before_count
    return df_final

def concat_delta_gross(name_list, total_frame, comparison_frame):
    col = ['name', 'movie', 'date', 'gross','net_gross_after', 'net_gross_before']
    concat = pd.DataFrame(columns = col)
    concat.set_index(['date'], inplace = True)
    for name in name_list:
        actor_frame = calc_comp_actor_df[calc_comp_actor_df['name'] == name]
        single_actor_gross = gross_by_day('1/1/1950', '12/12/2015', 365, actor_frame, name)
        #actor_frame.set_index(['date'], inplace = True)
        #df_new = pd.merge(left=actor_frame, right=single_actor_gross, how='left', left_index=True, right_index=True)
        col = ['date']
        comparison_dates = pd.DataFrame(columns = col)
        comparison_dates['date'] = comparison_frame.index
        df_new = pd.merge(left=comparison_dates, right=single_actor_gross, how='left', left_index=True, right_index=True)
        df_new['name'] = name
        concat = concat.append(df_new)
    return concat

comparison_nc_df = calc_comp_actor_df[calc_comp_actor_df['name'] == 'Nicolas Cage']
comparison_nc_df = comparison_nc_df['movie']
nc_dates = list(comparison_nc_df.index)
calc_comp_actor_df['NC Movie'] = calc_comp_actor_df[['movie']].isin(nc_dates)
#calc_comp_actor_df
list_of_actors = list(set(calc_comp_actor_df['name'].tolist()))    
df = concat_delta_gross(list_of_actors, calc_comp_actor_df, nc_dates)

/Users/johnwalters/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


TypeError: Argument 'left' has incorrect type (expected numpy.ndarray, got Index)

In [204]:
df

,date,gross,movie,name,net_gross_after,net_gross_before
date,,,,,,
0,10/3/14,NaN,NaN,Josh Hartnett,NaN,NaN
1,4/11/14,NaN,NaN,Josh Hartnett,NaN,NaN
2,3/22/13,NaN,NaN,Josh Hartnett,NaN,NaN
3,9/14/12,NaN,NaN,Josh Hartnett,NaN,NaN
4,3/16/12,NaN,NaN,Josh Hartnett,NaN,NaN
5,2/17/12,NaN,NaN,Josh Hartnett,NaN,NaN
6,10/14/11,NaN,NaN,Josh Hartnett,NaN,NaN
7,2/25/11,NaN,NaN,Josh Hartnett,NaN,NaN
8,1/7/11,NaN,NaN,Josh Hartnett,NaN,NaN


In [216]:
ts = pd.DataFrame(index=pd.date_range('1/1/1990', '1/1/2015')).sort_index(ascending = True)

In [215]:
ts1 = pd.DataFrame(index=ts.loc[nc_dates]

KeyError: "None of [[u'10/3/14', u'4/11/14', u'3/22/13', u'9/14/12', u'3/16/12', u'2/17/12', u'10/14/11', u'2/25/11', u'1/7/11', u'7/14/10', u'4/16/10', u'11/20/09', u'10/23/09', u'7/24/09', u'3/20/09', u'9/5/08', u'12/21/07', u'4/27/07', u'4/6/07', u'2/16/07', u'9/1/06', u'8/9/06', u'7/28/06', u'10/28/05', u'9/16/05', u'11/19/04', u'9/12/03', u'12/27/02', u'12/6/02', u'6/14/02', u'8/17/01', u'12/22/00', u'6/9/00', u'10/22/99', u'2/26/99', u'8/7/98', u'4/10/98', u'6/27/97', u'6/6/97', u'6/7/96', u'10/27/95', u'4/21/95', u'12/2/94', u'7/29/94', u'3/11/94', u'1/28/94', u'10/8/93', u'3/5/93', u'8/28/92', u'8/17/90', u'5/25/90', u'6/2/89', u'12/18/87', u'3/13/87', u'10/10/86', u'1/17/86', u'12/21/84', u'12/14/84', u'3/23/84', u'10/8/83', u'4/29/83', u'8/13/82']] are in the [index]"